# Introdução a Otimização Linear

**Técnica criada por George B. Dantzig em 1947 e que se propõe a otimizar (maximizar ou minimizar) o valor de uma função linear, respeitando um conjunto de restrições (equações ou inequações) lineares.**


> O principal solver de otimização linear do OR-Tools é o Glop, o sistema de programação linear do Google. É rápido, eficiente em termos de memória e numericamente estável. A próxima seção mostra como usar o Glop para resolver um problema linear simples.

## Um exemplo simples

A WYNDOR GLASS CO. fabrica produtos de vidro de alta qualidade, entre os quais janelas e portas de vidro. A empresa possui três fábricas industriais. As esquadrias de alumínio e ferragens são feitas na Fábrica 1, as esquadrias de madeira são produzidas na Fábrica 2 e, finalmente, a Fábrica 3 produz o vidro e monta os produtos.

Em conseqüência da queda nos ganhos, a direção decidiu modernizar a linha de produtos da empresa. Produtos não rentáveis estão sendo descontinuados, liberando capacidade produtiva para o lançamento de dois novos produtos com grande potencial de vendas:
- Produto 1: Uma porta de vidro de 2,5 m com esquadria de alumínio
- Produto 2: Uma janela duplamente adornada com esquadrias de madeira de 1,20 X 1,80 m

<img src="img/Fig3.14.png">

O produto 1 requer parte da capacidade produtiva das Fábricas 1 e 3, mas nenhuma da Fábrica 2. O produto 2 precisa apenas das Fábricas 2 e 3. 

A divisão de marketing concluiu que a empresa poderia vender tanto quanto fosse possível produzir desses produtos por essas fábricas. 

Entretanto, pelo fato de ambos os produtos poderem estar competindo pela mesma capacidade produtiva na Fábrica 3, não está claro qual mix dos dois produtos seria o mais lucrativo. 


**Maximize $Z = 3x_1 + 5x_2$ sujeito às seguintes restrições:**
\begin{equation}
x_1 	\le	4\\
2x_2 	\le	12\\
3x_1 + 2x_2	\le	18\\
\end{equation}

Tanto a função objetivo, $Z = 3x_1 + 5x_2$, quanto as restrições são dadas por expressões lineares, o que torna um problema linear.

As restrições definem a região viável, que é o triângulo mostrado abaixo, incluindo seu interior.

<img src="img/Fig4.1.png">

## As seções a seguir explicam como resolver o problema passo a passo.
0. Incluir as bibliotecas

In [ ]:
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

1. Escolha o solver.
> O código a seguir declara o solucionador. O MPsolver é um invólucro para vários solucionadores diferentes, incluindo o Glop. Escolha GLOP_LINEAR_PROGRAMMING para informar ao solucionador para usar Glop.

In [ ]:
MPSolver solver("ExemploLinear", MPSolver::GLOP_LINEAR_PROGRAMMING);

2. Crie as variáveis.
> crie variáveis x e y cujos valores estão no intervalo de 0 a infinito. Ou seja, x e y são variáveis não negativas.

In [ ]:
auto const infinity = solver.infinity();

auto const x1 = solver.MakeNumVar(0.0, infinity, "x1");
auto const x2 = solver.MakeNumVar(0.0, infinity, "x2");

std::cout << "Número de variáveis = " << solver.NumVariables();

3. Defina as restrições.
> defina as restrições das variáveis. Atribua a cada restrição um nome exclusivo (como restrição 0) e defina os coeficientes para a restrição.

In [ ]:
// x1 <= 4
auto const c0 = solver.MakeRowConstraint(-infinity, 4.0);
c0->SetCoefficient(x1, 1);

// 2*x2 <= 12
auto const c1 = solver.MakeRowConstraint(-infinity, 12.0);
c1->SetCoefficient(x2, 2);

// 3*x1 + 2*x2 <= 18
auto const c2 = solver.MakeRowConstraint(-infinity, 18.0);
c2->SetCoefficient(x1, 3);
c2->SetCoefficient(x2, 2);
std::cout << "Número de restrições = " << solver.NumConstraints();

4. Defina a função objetivo.
> O código a seguir define a função objetivo, $Z = 3x_1 + 5x_2$, e especifica que este é um problema de maximização.

In [ ]:
// FO: 3*x1 + 5*x2.
auto const objective = solver.MutableObjective();
objective->SetCoefficient(x1, 3);
objective->SetCoefficient(x2, 5);
objective->SetMaximization();

5. Execute o solver e exiba os resultados.
> O código a seguir chama o solver e exibe a solução.

In [ ]:
auto const result_status = solver.Solve();

if (result_status != MPSolver::OPTIMAL) {
  std::cout << "O problema não tem uma solução ótima!";
}

6. Exibir a solução
> O código a seguir exibe a solução.

In [ ]:
std::cout << "Solução:\n";
std::cout << "Valor ótimo da FO = " << objective->Value() << std::endl;
std::cout << x1->name() << " = " << x1->solution_value() << std::endl;
std::cout << x2->name() << " = " << x2->solution_value() << std::endl;

## O programa completo

Agora escreva o programa completo na caixa de código abaixo e verifique os resultados.

In [ ]:
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

{
    
}

O programa deve retornar a solução ótima para o problema, como mostrado abaixo.

Solução:
- $x_1 = 2$
- $x_2 = 6$

Valor ótimo: FO = 36

**A linha verde tracejada exibida no gráfico abaixo é encontrada definindo a função objetivo igual ao seu valor ótimo de 36.**

> Qualquer linha cuja equação tenha a forma **$3x_1 + 5x_2 = c$** é paralela à linha tracejada e 36 é o maior valor de **c** para o qual a linha cruza a região viável.

<img src="img/Fig4.9.png">

**Se você pensar na geometria no gráfico acima, em qualquer problema de otimização linear, pelo menos um vértice da região viável deve ser uma solução ótima.**

> Você poderia então criar um algoritmo para encontrar uma solução ótima percorrendo os vértices da região viável até que não haja mais melhorias na função objetivo. Essa é exatamente a idéia por trás do algoritmo **simplex**, o método mais usado para resolver problemas de otimização linear.



# O Método Simplex

1. Montar um dicionário inicial
2. Olhando a equação do z, escolha uma variável não-básica $x_{in}$ cujo aumento melhoraria a solução corrente do dicionário (coeficiente negativo se for minimização, positivo se for maximização). Se não houver tal variável, a solução corrente é ótima.
3. Calcule o máximo valor para que $x_{in}$ que não torne uma variável básica negativa. Se esse valor for infinito, o PL é ilimitado. Caso contrário, escolha uma variável $x_{out}$ que bloqueou o crescimento de $x_{in}$.
4. A variável $x_{in}$ entra na base, $x_{out}$ sai da base. Atualize o dicionário colocando $x_{in}$ isolado do lado esquerdo, $x_{out}$ vai pro lado direito. 
5. Volte para o Passo 2.
